In [4]:
import pandas as pd

In [10]:
df = pd.read_excel('data/data_entrenada_propia_GP.ods', engine='odf')
df

,Unnamed: 0,feedback,codigo,polaridad,Unnamed: 4
0,624927,"muy bien elaborada, se podría incluir ciertos ...",624927,P,NaN
1,543337,La información es ordenada y precisa pero reco...,543337,P,NaN
2,614284,Excelente aporte,614284,P,NaN
3,296130,"En la respuesta que le he puesto 3, ha sido po...",296130,N,NaN
4,555037,"Conciso, excelente",555037,P,NaN
...,...,...,...,...,...
2995,684953,excelente,684953,P,NaN
2996,444203,"Felicidades estimado/a estudiante, excelente t...",444203,P,NaN
2997,666824,Me gusto su presentación,666824,P,NaN
2998,167032,Se entiende en la formulacion de la encuesta q...,167032,P,NaN


In [11]:
# Revisión de datos nulos
print(df.isnull().sum())

Unnamed: 0       0
feedback         5
codigo           0
polaridad        7
Unnamed: 4    2792
dtype: int64


In [12]:
# Eliminar filas con valores nulos en 'feedback' o 'polaridad'
df = df.dropna(subset=['feedback', 'polaridad'])

In [13]:
# Verificar que los valores nulos se eliminaron
print(df.isnull().sum())

Unnamed: 0       0
feedback         0
codigo           0
polaridad        0
Unnamed: 4    2780
dtype: int64


In [14]:
# DataFrame solo con las columnas necesarias
df = df[['polaridad', 'feedback']]

In [15]:
# Revision de la columna 'polaridad'
print(df['polaridad'].value_counts())

polaridad
P         1780
N          552
NEU        330
NONE       322
NUE          2
NEUTRO       1
NOE          1
Name: count, dtype: int64


In [16]:
# Normalizar los nombres de la polaridad
df.loc[(df['polaridad'] == 'NEUTRO') | (df['polaridad'] == 'NUE') | (df['polaridad'] == 'NOE')]

,polaridad,feedback
68,NEUTRO,Muy Bien
253,NUE,Hay variedad de plantas que son medicinales de...
1331,NOE,10 puntos.
1446,NUE,Muy narrativa


In [17]:
# Cambiar valores incorrectos
df.loc[:, 'polaridad'] = df['polaridad'].replace({
    'NEUTRO': 'P',
    'NOE': 'NONE',
    'NUE': 'NEU'
})

In [18]:
# Revision de los cambios en la columna 'polaridad'
print(df['polaridad'].value_counts())

polaridad
P       1781
N        552
NEU      332
NONE     323
Name: count, dtype: int64


In [19]:
df

,polaridad,feedback
0,P,"muy bien elaborada, se podría incluir ciertos ..."
1,P,La información es ordenada y precisa pero reco...
2,P,Excelente aporte
3,N,"En la respuesta que le he puesto 3, ha sido po..."
4,P,"Conciso, excelente"
...,...,...
2995,P,excelente
2996,P,"Felicidades estimado/a estudiante, excelente t..."
2997,P,Me gusto su presentación
2998,P,Se entiende en la formulacion de la encuesta q...


In [20]:
print(df.dtypes)

polaridad    object
feedback     object
dtype: object


In [40]:
df["feedback"] = df["feedback"].astype(str).fillna("")

In [41]:
import polars as pl

df_polars = pl.from_pandas(df)

In [42]:
df_polars

polaridad,feedback
str,str
"""P""","""muy bien elaborada, se podría …"
"""P""","""La información es ordenada y p…"
"""P""","""Excelente aporte"""
"""N""","""En la respuesta que le he pues…"
"""P""","""Conciso, excelente"""
…,…
"""P""","""excelente"""
"""P""","""Felicidades estimado/a estudia…"
"""P""","""Me gusto su presentación"""


In [21]:
import requests
import time

In [22]:
OLLAMA_URL = "http://localhost:11434/api/generate"

In [23]:
def modelo(texto):
    prompt = f"""Clasifica el siguiente texto como Positivo (P), Negativo (N) o Neutro (NEU). Responde únicamente con una de las siguientes letras: P, N o NEU.

Texto: "{texto}"
Clasificación:"""

    response = requests.post(OLLAMA_URL, json={
        "model": MODEL_NAME,
        "prompt": prompt,
        "stream": False
    })

    if response.status_code == 200:
        return response.json()["response"].strip()
    else:
        print("Error:", response.text)
        return "Error"


In [24]:
def clasificador(df, cantidad):
    # Nuevo df con lacantidad de filas necesarias
    subset = df.head(cantidad).copy()
    # Inicio del conteo
    inicio = time.time()

    # Se agrega la polaridad según el modelo en columna "polaridad2"
    subset["polaridad2"] = subset["feedback"].apply(modelo)

    # Fin del conteo
    fin = time.time()
    duracion = fin - inicio

    return subset, duracion

In [45]:
def clasificador2(df, cantidad):

    # Tomar las primeras `cantidad` filas
    subset = df_polars.head(cantidad)

    # Medir solo el tiempo de clasificación
    inicio = time.time()

    resultado = subset.with_columns(
        pl.col("feedback").map_elements(modelo).alias("polaridad2")
    )

    duracion = time.time() - inicio

    return resultado, duracion

# Pruebas con LLaMA

In [25]:
MODEL_NAME = "llama3" 

## Prueba con 100 frases

In [27]:
df_100LLaMA, duracion = clasificador(df, 100)

In [28]:
print(duracion)

235.0589039325714


print(df_100)

In [29]:
df_100LLaMA

,polaridad,feedback,polaridad2
0,P,"muy bien elaborada, se podría incluir ciertos ...",P
1,P,La información es ordenada y precisa pero reco...,NEU
2,P,Excelente aporte,P
3,N,"En la respuesta que le he puesto 3, ha sido po...",N
4,P,"Conciso, excelente",P
...,...,...,...
95,NEU,Presentación y detalles.,NEU
96,NEU,"Responder en base a las preguntas, de forma má...",P
97,P,Muy bien,P
98,P,Buen trabajo,P


In [46]:
df_100LLaMA2, duracion2 = clasificador2(df_polars, 100)

C:\Users\jcesp\AppData\Local\Temp\ipykernel_18880\141178901.py:9: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  resultado = subset.with_columns(


In [47]:
print(duracion2)

243.07946848869324


In [48]:
df_100LLaMA2

polaridad,feedback,polaridad2
str,str,str
"""P""","""muy bien elaborada, se podría …","""NEU"""
"""P""","""La información es ordenada y p…","""NEU"""
"""P""","""Excelente aporte""","""P"""
"""N""","""En la respuesta que le he pues…","""NEU"""
"""P""","""Conciso, excelente""","""NEU"""
…,…,…
"""NEU""","""Presentación y detalles.""","""NEU"""
"""NEU""","""Responder en base a las pregun…","""P"""
"""P""","""Muy bien""","""P"""


# Pruebas con Qwen

In [32]:
MODEL_NAME = "qwen" 

In [33]:
df_100Qwen, duracion = clasificador(df, 100)

In [34]:
print(duracion)

231.64854311943054


In [35]:
df_100Qwen

,polaridad,feedback,polaridad2
0,P,"muy bien elaborada, se podría incluir ciertos ...",NEU
1,P,La información es ordenada y precisa pero reco...,NEU
2,P,Excelente aporte,P
3,N,"En la respuesta que le he puesto 3, ha sido po...",NEU
4,P,"Conciso, excelente",Positivo (P)
...,...,...,...
95,NEU,Presentación y detalles.,NEU
96,NEU,"Responder en base a las preguntas, de forma má...",NEU
97,P,Muy bien,P
98,P,Buen trabajo,P


# Comparación de Resultados

In [99]:
df1 = df.head(100)

In [101]:
pd.set_option('display.max_colwidth', None)
df1

,polaridad,feedback
0,P,"muy bien elaborada, se podría incluir ciertos detalles previos a embotellar el vino para obtener un vino mas limpio"
1,P,La información es ordenada y precisa pero recomendaría resumirla buscando contenido mas preciso
2,P,Excelente aporte
3,N,"En la respuesta que le he puesto 3, ha sido porque el significado de su oración no es la correcta, por lo tanto da ha entender algo erróneo."
4,P,"Conciso, excelente"
...,...,...
95,NEU,Presentación y detalles.
96,NEU,"Responder en base a las preguntas, de forma más puntual y en base a la asignatura en estudio."
97,P,Muy bien
98,P,Buen trabajo


In [115]:
# Cambiar valores incorrectos
df_100.loc[:, 'polaridad'] = df_100['polaridad'].replace({
    'Positivo.': 'P',
    'Negativo': 'N',
    'Neutro': 'NEU',
    'Positiva': 'P',
    'Negativo.': 'N'
})

In [116]:
print(df_100['polaridad'].value_counts())

polaridad
P      60
NEU    20
N      20
Name: count, dtype: int64


In [117]:
df_100

,polaridad,feedback,diferencia
0,P,"muy bien elaborada, se podría incluir ciertos detalles previos a embotellar el vino para obtener un vino mas limpio",0
1,NEU,La información es ordenada y precisa pero recomendaría resumirla buscando contenido mas preciso,1
2,P,Excelente aporte,0
3,N,"En la respuesta que le he puesto 3, ha sido porque el significado de su oración no es la correcta, por lo tanto da ha entender algo erróneo.",0
4,P,"Conciso, excelente",0
...,...,...,...
95,NEU,Presentación y detalles.,0
96,NEU,"Responder en base a las preguntas, de forma más puntual y en base a la asignatura en estudio.",0
97,P,Muy bien,0
98,P,Buen trabajo,0


In [118]:
contador = 0

for p1, p2 in zip(df1['polaridad'], df_100['polaridad']):
    if p1 == p2:
        contador += 1

print(contador)

61


In [119]:
print(df_100.iloc[27])  # Imprime la fila en la posición 5 (sexta fila, porque empieza en 0)

polaridad                              N
feedback      No se cargó la infografía.
diferencia                             1
Name: 27, dtype: object


In [120]:
comparacion = df1['polaridad'] != df_100['polaridad']

df1['diferencia'] = comparacion.astype(int)
df_100['diferencia'] = comparacion.astype(int)

C:\Users\jcesp\AppData\Local\Temp\ipykernel_27152\231996312.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['diferencia'] = comparacion.astype(int)


In [121]:
df_f = df1[df1['diferencia'] == 1]
df_f2 = df_100[df_100['diferencia'] == 1]

In [122]:
df2 = df_f2.rename(columns=lambda col: f"{col}_2")

In [123]:
df2

,polaridad_2,feedback_2,diferencia_2
1,NEU,La información es ordenada y precisa pero recomendaría resumirla buscando contenido mas preciso,1
10,N,Debió seguir las instrucciones del docente y subir el ensayo en formato jpg y en la sección de respuesta solo debía ir el titulo del ensayo no todo el texto en si. Además sobrepasa el limite de palabras estipuladas ya que consta con 236. La imagen no refleja la relación entre el recurso del suelo y la carrera de ingeniería ambiental. Por ultimo las dos referencias utilizadas solo hablan de las ventajas de estudiar Ingeniería ambiental mas no la relación entre el suelo y dicha carrera.,1
11,P,Podria mejor graficamente y ser mas facil de leer para un futuro,1
12,N,"No puedo calificar porque la imagen no corresponde, la persona se equivocó al subir.",1
18,N,no hay archivo o no se abre,1
20,N,El video es del año 2014 y no corresponde a la tarea solicitada,1
21,N,bien hechop,1
22,P,Excelent,1
23,NEU,"No utiliza referencias bibliográficas, pero para mi estan bien sus puntos de vista, claros y precisos",1
26,NEU,El orden de cada proceso esta bien estructurado.Pero deberia tener al menos un grafico o un párrafo explicativo.,1


In [124]:
df_combinado = pd.concat([df_f, df2], axis=1)

In [125]:
df_combinado

,polaridad,feedback,diferencia,polaridad_2,feedback_2,diferencia_2
1,P,La información es ordenada y precisa pero recomendaría resumirla buscando contenido mas preciso,1,NEU,La información es ordenada y precisa pero recomendaría resumirla buscando contenido mas preciso,1
10,NEU,Debió seguir las instrucciones del docente y subir el ensayo en formato jpg y en la sección de respuesta solo debía ir el titulo del ensayo no todo el texto en si. Además sobrepasa el limite de palabras estipuladas ya que consta con 236. La imagen no refleja la relación entre el recurso del suelo y la carrera de ingeniería ambiental. Por ultimo las dos referencias utilizadas solo hablan de las ventajas de estudiar Ingeniería ambiental mas no la relación entre el suelo y dicha carrera.,1,N,Debió seguir las instrucciones del docente y subir el ensayo en formato jpg y en la sección de respuesta solo debía ir el titulo del ensayo no todo el texto en si. Además sobrepasa el limite de palabras estipuladas ya que consta con 236. La imagen no refleja la relación entre el recurso del suelo y la carrera de ingeniería ambiental. Por ultimo las dos referencias utilizadas solo hablan de las ventajas de estudiar Ingeniería ambiental mas no la relación entre el suelo y dicha carrera.,1
11,NONE,Podria mejor graficamente y ser mas facil de leer para un futuro,1,P,Podria mejor graficamente y ser mas facil de leer para un futuro,1
12,NEU,"No puedo calificar porque la imagen no corresponde, la persona se equivocó al subir.",1,N,"No puedo calificar porque la imagen no corresponde, la persona se equivocó al subir.",1
18,NONE,no hay archivo o no se abre,1,N,no hay archivo o no se abre,1
20,NONE,El video es del año 2014 y no corresponde a la tarea solicitada,1,N,El video es del año 2014 y no corresponde a la tarea solicitada,1
21,P,bien hechop,1,N,bien hechop,1
22,NONE,Excelent,1,P,Excelent,1
23,P,"No utiliza referencias bibliográficas, pero para mi estan bien sus puntos de vista, claros y precisos",1,NEU,"No utiliza referencias bibliográficas, pero para mi estan bien sus puntos de vista, claros y precisos",1
26,NONE,El orden de cada proceso esta bien estructurado.Pero deberia tener al menos un grafico o un párrafo explicativo.,1,NEU,El orden de cada proceso esta bien estructurado.Pero deberia tener al menos un grafico o un párrafo explicativo.,1


In [128]:
df_final = df_combinado[df_combinado['polaridad'] != "NONE"]

In [134]:
print(len(df_final))

19


In [135]:
df_final

,polaridad,feedback,diferencia,polaridad_2,feedback_2,diferencia_2
1,P,La información es ordenada y precisa pero recomendaría resumirla buscando contenido mas preciso,1,NEU,La información es ordenada y precisa pero recomendaría resumirla buscando contenido mas preciso,1
10,NEU,Debió seguir las instrucciones del docente y subir el ensayo en formato jpg y en la sección de respuesta solo debía ir el titulo del ensayo no todo el texto en si. Además sobrepasa el limite de palabras estipuladas ya que consta con 236. La imagen no refleja la relación entre el recurso del suelo y la carrera de ingeniería ambiental. Por ultimo las dos referencias utilizadas solo hablan de las ventajas de estudiar Ingeniería ambiental mas no la relación entre el suelo y dicha carrera.,1,N,Debió seguir las instrucciones del docente y subir el ensayo en formato jpg y en la sección de respuesta solo debía ir el titulo del ensayo no todo el texto en si. Además sobrepasa el limite de palabras estipuladas ya que consta con 236. La imagen no refleja la relación entre el recurso del suelo y la carrera de ingeniería ambiental. Por ultimo las dos referencias utilizadas solo hablan de las ventajas de estudiar Ingeniería ambiental mas no la relación entre el suelo y dicha carrera.,1
12,NEU,"No puedo calificar porque la imagen no corresponde, la persona se equivocó al subir.",1,N,"No puedo calificar porque la imagen no corresponde, la persona se equivocó al subir.",1
21,P,bien hechop,1,N,bien hechop,1
23,P,"No utiliza referencias bibliográficas, pero para mi estan bien sus puntos de vista, claros y precisos",1,NEU,"No utiliza referencias bibliográficas, pero para mi estan bien sus puntos de vista, claros y precisos",1
32,NEU,"El cultivo de aji una opcion muy buena, ya que poestrior a ello y cuando ya obtenga frutos puede elaborar insecticidas y aplicas en su huerto.",1,P,"El cultivo de aji una opcion muy buena, ya que poestrior a ello y cuando ya obtenga frutos puede elaborar insecticidas y aplicas en su huerto.",1
38,NEU,Le falta mejorar con su lectura y análisis de los dos casos.,1,N,Le falta mejorar con su lectura y análisis de los dos casos.,1
58,NEU,Puedo ver que el tema es de gran interés para la compañera,1,P,Puedo ver que el tema es de gran interés para la compañera,1
60,N,La imagen asociada no se puede visualizar,1,NEU,La imagen asociada no se puede visualizar,1
65,NEU,"Podría mejorar e investigar sobre los cursos que se ofertan a los emprendedores y emprendedoras, podría indagar sobre las inversiones que está haciendo CRISFE, FUDELA, CODEIS, PNUD, entre otras.Señala que el sector privado no ofrece cursos, sin embargo, el equipo 2x2 MKT , Mercatitlan, y otros si lo ofrecen. Son cursos digitales, y por ende tienen ese alcance nacional Saludos",1,P,"Podría mejorar e investigar sobre los cursos que se ofertan a los emprendedores y emprendedoras, podría indagar sobre las inversiones que está haciendo CRISFE, FUDELA, CODEIS, PNUD, entre otras.Señala que el sector privado no ofrece cursos, sin embargo, el equipo 2x2 MKT , Mercatitlan, y otros si lo ofrecen. Son cursos digitales, y por ende tienen ese alcance nacional Saludos",1
